<a href="https://colab.research.google.com/github/ajitkumarkp/ML/blob/master/ALPR_YOLO_V_LP_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/ALPR/darknet

/content/drive/My Drive/ALPR/darknet


# ALPR has following steps:

1.   Vehicle detection: Using YOLO trained on PASCAL-VOC dataset
2.   LP detection: Using YOLO trained on OpenALPR dataset
3.   OCR: Using YOLO trained specfically on a dataset of LPs only

# 1. Vehicle detection

Download weights and config file of YOLOv2 network trained on PASCAL-VOC dataset from http://www.inf.ufrgs.br/~smsilva/alpr-unconstrained/data/vehicle-detector/, by S. M. Silva author of License Plate Detection and Recognition in Unconstrained Scenarios

In [0]:
!wget -c -N www.inf.ufrgs.br/~smsilva/alpr-unconstrained/data/vehicle-detector/yolo-voc.cfg     -P vehicle-detector/
!wget -c -N www.inf.ufrgs.br/~smsilva/alpr-unconstrained/data/vehicle-detector/voc.data         -P vehicle-detector/
!wget -c -N www.inf.ufrgs.br/~smsilva/alpr-unconstrained/data/vehicle-detector/yolo-voc.weights -P vehicle-detector/
!wget -c -N www.inf.ufrgs.br/~smsilva/alpr-unconstrained/data/vehicle-detector/voc.names        -P vehicle-detector/

--2020-04-24 16:20:58--  http://www.inf.ufrgs.br/~smsilva/alpr-unconstrained/data/vehicle-detector/yolo-voc.cfg
Resolving www.inf.ufrgs.br (www.inf.ufrgs.br)... 143.54.11.34
Connecting to www.inf.ufrgs.br (www.inf.ufrgs.br)|143.54.11.34|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2721 (2.7K) [text/plain]
Saving to: ‘vehicle-detector/yolo-voc.cfg’

yolo-voc.cfg        100%[===================>]   2.66K  --.-KB/s    in 0s      

2020-04-24 16:20:59 (224 MB/s) - ‘vehicle-detector/yolo-voc.cfg’ saved [2721/2721]

--2020-04-24 16:21:01--  http://www.inf.ufrgs.br/~smsilva/alpr-unconstrained/data/vehicle-detector/voc.data
Resolving www.inf.ufrgs.br (www.inf.ufrgs.br)... 143.54.11.34
Connecting to www.inf.ufrgs.br (www.inf.ufrgs.br)|143.54.11.34|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69 [text/plain]
Saving to: ‘vehicle-detector/voc.data’

voc.data            100%[===================>]      69  --.-KB/s    in 0s      

2020-04-24 16

Utility functions:

getOutputsNames: Get the names of the output layers for given input neural network.

postprocess: to get rid of detected bounding box with low confidence

drawPred: to draw the predicted bounding box

crop_region: to crop out specified region from given input image

We also define Label a bounding box class. All detected bounding boxs are stored as an object of this class.

In [0]:
def getOutputsNames(net):
  """ Get the names of the output layers.
  
  Generally in a sequential CNN network there will be 
  only one output layer at the end. In the YOLOv3 
  architecture, there are multiple output layers giving
  out predictions. This function gives the names of the 
  output layers. An output layer is not connected to 
  any next layer.
  
  Args
    net : neural network
  """
  # Get the names of all the layers in the network
  layersNames = net.getLayerNames()
  # Get the names of the output layers, i.e. the layers with unconnected outputs
  return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]


In [0]:
import cv2 as cv
# Remove the bounding boxes with low confidence using non-maxima suppression
def postprocess(frame, outs, confThreshold, nmsThreshold=0.4):
  frameHeight = frame.shape[0]
  frameWidth = frame.shape[1]

  classIds = []
  confidences = []
  boxes = []
  # Scan through all the bounding boxes output from the network and keep only the
  # ones with high confidence scores. Assign the box's class label as the class with the highest score.
  classIds = []
  confidences = []
  boxes = []
  predictions = []

  for out in outs:
    for detection in out:
      scores = detection[5:]
      classId = np.argmax(scores)
      confidence = scores[classId]
      if confidence > confThreshold:
        center_x = int(detection[0] * frameWidth)
        center_y = int(detection[1] * frameHeight)
        width = int(detection[2] * frameWidth)
        height = int(detection[3] * frameHeight)
        left = int(center_x - width / 2)
        top = int(center_y - height / 2)
        classIds.append(classId)
        confidences.append(float(confidence))
        boxes.append([left, top, width, height])

  # Perform non maximum suppression to eliminate redundant overlapping boxes with
  # lower confidences.
  if nmsThreshold:
    indices = cv.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
  else:
    indices = [[x] for x in range(len(boxes))]

  for i in indices:
    i = i[0]
    box = boxes[i]
    left = box[0]
    top = box[1]
    width = box[2]
    height = box[3]
    predictions.append([classIds[i], confidences[i], [left, top, left + width, top + height]])

  return predictions

In [0]:
import cv2 as cv
# Draw the predicted bounding box
def drawPred(frame, pred):
  classId = pred[0]
  conf = pred[1]
  box = pred[2]
  left, top, right, bottom =  box[0], box[1], box[2], box[3]
  # draw bounding box
  cv.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 3)

In [0]:
import numpy as np

class Label:

	def __init__(self,cl=-1,tl=np.array([0.,0.]),br=np.array([0.,0.]),prob=None):
		self.__tl 	= tl
		self.__br 	= br
		self.__cl 	= cl
		self.__prob = prob

	def __str__(self):
		return 'Class: %d, top_left(x:%f,y:%f), bottom_right(x:%f,y:%f)' % (self.__cl, self.__tl[0], self.__tl[1], self.__br[0], self.__br[1])

	def copy(self):
		return Label(self.__cl,self.__tl,self.__br)

	def wh(self): return self.__br-self.__tl

	def cc(self): return self.__tl + self.wh()/2

	def tl(self): return self.__tl
 
	def br(self): return self.__br

	def tr(self): return np.array([self.__br[0],self.__tl[1]])

	def bl(self): return np.array([self.__tl[0],self.__br[1]])

	def cl(self): return self.__cl

	def area(self): return np.prod(self.wh())

	def prob(self): return self.__prob

	def set_class(self,cl):
		self.__cl = cl

	def set_tl(self,tl):
		self.__tl = tl

	def set_br(self,br):
		self.__br = br

	def set_wh(self,wh):
		cc = self.cc()
		self.__tl = cc - .5*wh
		self.__br = cc + .5*wh

	def set_prob(self,prob):
		self.__prob = prob

def crop_region(I,label,bg=0.5):

  wh = np.array(I.shape[1::-1])

  ch = I.shape[2] if len(I.shape) == 3 else 1
  tl = np.floor(label.tl()*wh).astype(int)
  br = np.ceil (label.br()*wh).astype(int)
  outwh = br-tl

  if np.prod(outwh) == 0.:
    return None

  outsize = (outwh[1],outwh[0],ch) if ch > 1 else (outwh[1],outwh[0])
  if (np.array(outsize) < 0).any():
    pause()
  Iout  = np.zeros(outsize,dtype=I.dtype) + bg

  offset  = np.minimum(tl,0)*(-1)
  tl      = np.maximum(tl,0)
  br      = np.minimum(br,wh)
  wh      = br - tl

  Iout[offset[1]:(offset[1] + wh[1]),offset[0]:(offset[0] + wh[0])] = I[tl[1]:br[1],tl[0]:br[0]]

  return Iout

vehicle_detection function: Takes an image as input and returns Icars list of cropped images of vehicles as well as Lcars list of bouding box around vehicles.

postprocess function: To get rid of detected bounding box with low confidence. The postprocess utility function internally uses cv.dnn.NMSBoxes which perform non maximum suppression to eliminate redundant overlapping boxes with lower confidences. We keep only those bounding boxs whose corresponding classId is either car (class number 6) or bus (class number 7), since this two classId are related to vehicles.

In [0]:
import cv2 as cv
import numpy as np

# Initialize the parameters
vehicle_threshold = .5
vehicle_weights = 'vehicle-detector/yolo-voc.weights'
vehicle_netcfg  = 'vehicle-detector/yolo-voc.cfg'

# Load the model
vehicle_net = cv.dnn.readNetFromDarknet(vehicle_netcfg, vehicle_weights)
vehicle_net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
vehicle_net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

def vehicle_detection(frame):

  # Create a 4D blob from a frame.
  blob = cv.dnn.blobFromImage(frame, 1/255, (416, 416), [0,0,0], 1, crop=False)

  # Sets the input to the network
  vehicle_net.setInput(blob)

  # Runs the forward pass to get output of the output layers
  outs = vehicle_net.forward(getOutputsNames(vehicle_net))

  # Remove the bounding boxes with low confidence
  R = postprocess(frame, outs, vehicle_threshold)
  
  Icars = []
  Lcars = []

  if len(R):
    WH = np.array(frame.shape[1::-1], dtype=float)
    for i, r in enumerate(R):
      # if classId in ['car', 'bus'] and confidence > vehicle_threshold
      if r[0] in [6, 7] and r[1] > vehicle_threshold:
        box = r[2]
        x1,y1,x2,y2 = (np.array(r[2])/np.concatenate((WH,WH))).tolist()
        tl = np.array([x1, y1])
        br = np.array([x2, y2])
        label = Label(0,tl,br)
        Lcars.append(label)
        Icar = crop_region(frame,label)
        Icars.append(Icar.astype(np.uint8))

  return Icars, Lcars

# 2. LP detection 

For each detected vehicle in the first step, we apply this second step to detect region of license plate in detected vehicle region.

lp_detection function: Takes vehicle image as input and returns Llps list of bounding boxs around detected license plates and Ilps list of cropped images of detected license plates.

Again, we use postprocess utility function to remove bouding box with low threshold.

In [0]:
import cv2 as cv
import numpy as np
import time

# Initialize the parameters
lp_threshold = .6
yolo_lp_confi_path = 'yolov3-LP-test.cfg'
yolo_lp_weights_path = 'backup/yolov3-LP-train_best.weights'

# load the network
lp_net = cv.dnn.readNetFromDarknet(yolo_lp_confi_path, yolo_lp_weights_path)
lp_net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
lp_net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

def lp_detection(vehicle_img):
  # Create a 4D blob from a frame.
  blob = cv.dnn.blobFromImage(vehicle_img, 1/255, (416, 416), [0,0,0], 1, crop=False)
  # Sets the input to the network
  lp_net.setInput(blob)
  # Runs the forward pass to get output of the output layers
  start 	= time.time()
  outs = lp_net.forward(getOutputsNames(lp_net))
  elapsed = time.time() - start
  # Remove the bounding boxes with low confidence
  R = postprocess(vehicle_img, outs, lp_threshold)

  Llps = []
  Ilps = []

  if len(R):
    WH = np.array(vehicle_img.shape[1::-1], dtype=float)
    for i, r in enumerate(R):
      if r[1] > lp_threshold:
        box = r[2]
        x1,y1,x2,y2 = (np.array(r[2])/np.concatenate((WH,WH))).tolist()
        tl = np.array([x1, y1])
        br = np.array([x2, y2])
        label = Label(0,tl,br)
        Llps.append(label)
        Ilp = crop_region(vehicle_img,label)
        Ilps.append(Ilp.astype(np.uint8))

  return Llps, Ilps, elapsed

# 3. OCR

OCR is third and last step in ALPR system. In this step, for each detected license plate we apply OCR for a) character segmentation and b) character recognition.

We will perform character segmentation and recognition over the detected LP using a Modified YOLO network, with the same architecture presented in Real-Time Brazilian License Plate Detection and Recognition Using Deep Convolutional Neural Networks by S. M. Silva and C. R. Jung.

The network structure of Modified YOLO:

We will use weights and config file of Modified YOLO from here, which is same as used by S. M. Silva an author of License Plate Detection and Recognition in Unconstrained Scenarios. The training dataset for Modified YOLO was considerably enlarged by using synthetic and augmented data to cope with LP characteristics of different regions around the world (Europe, United States and Brazil).

The artificially created data consist of pasting a string of seven characters onto a textured background and then performing random transformations, such as rotation, translation, noise, and blur.


Download weights and config file of pretrained Modified YOLO

'/content/drive/My Drive/ALPR/darknet'

In [11]:
!wget -c -N www.inf.ufrgs.br/~smsilva/alpr-unconstrained/data/ocr/ocr-net.cfg     -P ocr/
!wget -c -N www.inf.ufrgs.br/~smsilva/alpr-unconstrained/data/ocr/ocr-net.names   -P ocr/
!wget -c -N www.inf.ufrgs.br/~smsilva/alpr-unconstrained/data/ocr/ocr-net.weights -P ocr/
!wget -c -N www.inf.ufrgs.br/~smsilva/alpr-unconstrained/data/ocr/ocr-net.data    -P ocr/


--2020-04-25 18:46:19--  http://www.inf.ufrgs.br/~smsilva/alpr-unconstrained/data/ocr/ocr-net.cfg
Resolving www.inf.ufrgs.br (www.inf.ufrgs.br)... 143.54.11.34
Connecting to www.inf.ufrgs.br (www.inf.ufrgs.br)|143.54.11.34|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1573 (1.5K) [text/plain]
Saving to: ‘ocr/ocr-net.cfg’

ocr-net.cfg         100%[===================>]   1.54K  --.-KB/s    in 0s      

2020-04-25 18:46:20 (105 MB/s) - ‘ocr/ocr-net.cfg’ saved [1573/1573]

--2020-04-25 18:46:21--  http://www.inf.ufrgs.br/~smsilva/alpr-unconstrained/data/ocr/ocr-net.names
Resolving www.inf.ufrgs.br (www.inf.ufrgs.br)... 143.54.11.34
Connecting to www.inf.ufrgs.br (www.inf.ufrgs.br)|143.54.11.34|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70 [text/plain]
Saving to: ‘ocr/ocr-net.names’

ocr-net.names       100%[===================>]      70  --.-KB/s    in 0s      

2020-04-25 18:46:21 (5.41 MB/s) - ‘ocr/ocr-net.names’ saved [70/70]

--

dknet_label_conversion: To convert detected character bounding box into an object of Label class.

In [0]:
#@title
# Import necessary modules
import numpy as np

def dknet_label_conversion(R,img_width,img_height, classes):
	WH = np.array([img_width,img_height],dtype=float)
	L  = []
	for r in R:
		center = np.array(r[2][:2])/WH
		wh2 = (np.array(r[2][2:])/WH)*.5
		L.append(Label(ord(classes[r[0]]),tl=center-wh2,br=center+wh2,prob=r[1]))
	return L

lp_ocr function: Takes image of license plate as input, apply Modified YOLO OCR, converts license plate text into string and return that string as output.

In [0]:
import cv2 as cv

# Initialize the parameters
ocr_threshold = .4
ocr_weights_path = 'ocr/ocr-net.weights'
ocr_confi_path   = 'ocr/ocr-net.cfg'
ocr_classes_path = 'ocr/ocr-net.names'
ocr_classes = None

with open(ocr_classes_path, 'rt') as f:
  ocr_classes = f.read().rstrip('\n').split('\n')

# load the network
ocr_net = cv.dnn.readNetFromDarknet(ocr_confi_path, ocr_weights_path)
ocr_net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
ocr_net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

def lp_ocr(lp_img):
  h,w,_ = lp_img.shape
  # Create a 4D blob from a frame
  blob = cv.dnn.blobFromImage(lp_img, 1/255, (240, 80), [0,0,0], 1, crop=False)
  # Sets the input to the network
  ocr_net.setInput(blob)
  # Runs the forward pass to get output of the output layers
  outs = ocr_net.forward(getOutputsNames(ocr_net))
  # Remove the bounding boxes with low confidence
  R = postprocess(lp_img, outs, ocr_threshold, None)

  lp_str = ''
  if len(R):
    L = dknet_label_conversion(R, w, h, ocr_classes)
    L.sort(key=lambda x: x.tl()[0])    
    lp_str = ''.join([chr(l.cl()) for l in L])

  return lp_str

# 4. Inference 

We have already defined 1) vehicle_detection 2) lp_detection and 3) lp_ocr to perform vehicle detection, license plate detection and OCR respectively.

Let's implement our ALPR systen which is compised all these three function in a sequential pipeline.

we define few utility functions like,

draw_label: to draw bounding box using object of Label class as input

draw_losangle: to draw bouding box using a set of four corner points.

write2img: to write text on a input image around given bounding box.

In [14]:
!wget "https://raw.githubusercontent.com/sergiomsilva/alpr-unconstrained/master/samples/test/03066.jpg" -O test_img.jpg

--2020-04-25 18:55:53--  https://raw.githubusercontent.com/sergiomsilva/alpr-unconstrained/master/samples/test/03066.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160770 (157K) [image/jpeg]
Saving to: ‘test_img.jpg’

test_img.jpg        100%[===================>] 157.00K  --.-KB/s    in 0.04s   

2020-04-25 18:55:53 (3.78 MB/s) - ‘test_img.jpg’ saved [160770/160770]



In [0]:
import numpy as np
import cv2 as cv

def draw_label(I,l,color=(255,0,0),thickness=1):
	wh = np.array(I.shape[1::-1]).astype(float)
	tl = tuple((l.tl()*wh).astype(int).tolist())
	br = tuple((l.br()*wh).astype(int).tolist())
	cv.rectangle(I,tl,br,color,thickness=thickness)

def draw_losangle(I,pts,color=(1.,1.,1.),thickness=1):
	assert(pts.shape[0] == 2 and pts.shape[1] == 4)

	for i in range(4):
		pt1 = tuple(pts[:,i].astype(int).tolist())
		pt2 = tuple(pts[:,(i+1)%4].astype(int).tolist())
		cv.line(I,pt1,pt2,color,thickness)

def write2img(Img,label,strg,txt_color=(0,0,0),bg_color=(255,255,255),font_size=1):
	wh_img = np.array(Img.shape[1::-1])

	font = cv.FONT_HERSHEY_SIMPLEX

	wh_text,v = cv.getTextSize(strg, font, font_size, 3)
	bl_corner = label.tl()*wh_img

	tl_corner = np.array([bl_corner[0],bl_corner[1]-wh_text[1]])/wh_img
	br_corner = np.array([bl_corner[0]+wh_text[0],bl_corner[1]])/wh_img
	bl_corner /= wh_img

	if (tl_corner < 0.).any():
		delta = 0. - np.minimum(tl_corner,0.)
	elif (br_corner > 1.).any():
		delta = 1. - np.maximum(br_corner,1.)
	else:
		delta = 0.

	tl_corner += delta
	br_corner += delta
	bl_corner += delta

	tpl = lambda x: tuple((x*wh_img).astype(int).tolist())

	cv.rectangle(Img, tpl(tl_corner), tpl(br_corner), bg_color, -1)	
	cv.putText(Img,strg,tpl(bl_corner),font,font_size,txt_color,3)

To infer on the test image we apply

first, vehicle_detection to detect all vehicles in an input test image. Output of this step is Icars a list of cropped vehicles regions as well as Lcars a list of bounding boxs around detected vehicles.

second, lp_detection on each cropped vehicle regions in Icars to detect license plates. Output of this step is Llps a list of bouding boxs around detected license plate as well as Ilps cropped images of license plate in given vehicle image.

third, lo_ocr on each cropped license plate region in Ilps to convert licese plat text in it to a string.

finally, write2img to write recognized license plat charactes on the input test image.

In [21]:
# Import necessary modules
import numpy as np
import cv2 as cv
# read test image
test_img = cv.imread('test_img.jpg')
# detect cars
Icars, Lcars = vehicle_detection(test_img)
print('# vehicle detected: {}'.format(len(Icars)))
# for each detected car in test image
for Icar, Lcar in zip(Icars, Lcars):
  # draw car bounding box on test image
  draw_label(test_img,Lcar,color=(0,255,255),thickness=3)
  # detect LP in detected car
  Llps, Ilps, elapsed = lp_detection(Icar)   
  # for each detected LP in the detected car image
  for Llp, Ilp in zip(Llps, Ilps):
    # draw LP bounding box on test image
    tl = Llp.tl() * Lcar.wh() + Lcar.tl()
    br = Llp.br() * Lcar.wh() + Lcar.tl()
    draw_label(test_img,Label(0,tl,br),color=(255,0,0),thickness=3)
    # Recognize characters
    lp_str = lp_ocr(Ilp)
    # write text on test image
    write2img(test_img,Label(0,tl,br),lp_str)

# vehicle detected: 3


In [22]:
# Import necessary modules
import matplotlib
import matplotlib.pyplot as plt
# Display inference
fig=plt.figure(figsize=(10, 10))
plt.imshow(test_img[:,:,::-1])
plt.show()

# 5. Conclusion

There is improvement in performance of text recognition for Modified YOLO as compared to tesseract

Even now, if the images are given in unconstrained scenarios like more rotated and different view of the car's license plate, our YOLO-Net fails to give accurate bounding box around license plate which may affect the performace of OCR